<center><h1><b>Advacned RAG - Strategies</b></h1></center>

## Advanced Retrieval and Reranking

In [1]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context
)


ImportError: cannot import name 'ChatMessage' from 'llama_index.core.llms' (unknown location)

In [ ]:

# BM25 retriever
retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2
)

from llama_index.core.response.notebook_utils import display_source_node
nodes = retriever.retrieve(prompt)
for node in nodes:
    display_source_node(node)

In [ ]:
# router retriever
from llama_index.core.tools import RetrieverTool

vector_retriever = VectorIndexRetriever(index)
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2
)

retriever_tools = [
    RetrieverTool.from_defaults(
        retriever=vector_retriever,
        description='useful in most cases',
    ),
    RetrieverTool.from_defaults(
        retriever=bm25_retriever,
        description='useful for searching about specific information',
    ),
]

from llama_index.core.retrievers import RouterRetriever

retriever = RouterRetriever.from_defaults(
    retriever_tools=retriever_tools,
    llm=llm,
    select_multi=True,
)

nodes = retriever.retrieve(prompt)
for node in nodes:
    display_source_node(node)

In [ ]:
# hybrid retriever + reranking

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

index = VectorStoreIndex(
    nodes,
    storage_context=storage_context
)

vector_retriever = index.as_retriever(similarity_top_k=10)

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=10
)

from llama_index.core.retrievers import BaseRetriever

class Hybridretriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        super().__init__()

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retriever(query, **kwargs)
        vector_nodes = self.vector_retriever.retriever(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        nodes_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        
        return all_nodes
    

index.as_retriever(similarity_top_k=5)
hybrid_retriever = Hybridretriever(vector_retriever, bm25_retriever)

# re-ranker
from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(
    top_n=4,
    model='BAAI/bge-reranker-base'
)

from llama_index.core import QueryBundle
retrieved_nodes = hybrid_retriever.retrieve(prompt)

reranked_nodes = reranker.postprocess_nodes(
    nodes,
    query_bundle = QueryBundle(prompt)
)

from llama_index.core.response.notebook_utils import display_source_node

for node in reranked_nodes:
    display_source_node(node)

In [ ]:
# query engine
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[reranker],
    llm=llm,
    streaming=True
)

response = query_engine.query(prompt)
response.print_response_stream()

## Instructor - instruction-tuned embedding model

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("hkunlp/instructor-large")

## News article scraping - `newspaper3k`

In [74]:
import newspaper

In [147]:
ny = newspaper.build('https://www.nytimes.com/2024/', memoize_articles=False)
ny.size()

183

In [148]:
ny.article_urls()

['https://www.nytimes.com/section/business/media',
 'https://www.nytimes.com/interactive/2023/10/27/arts/music/music-fivemins-collection.html',
 'https://www.nytimes.com/2023/10/19/podcasts/serial-kids-rutherford-county.html',
 'https://www.nytimes.com/article/submit-crossword-puzzles-the-new-york-times.html',
 'https://www.nytimes.com/2022/09/19/crosswords/mini-to-maestro-part-1.html',
 'https://theathletic.com/5029170/2023/11/06/free-agents-mlb-list/',
 'https://www.nytimes.com/2024/02/14/world/asia/pakistan-election-coalition-government.html',
 'https://www.nytimes.com/2024/02/13/us/politics/mayorkas-impeachment-house.html',
 'https://www.nytimes.com/2024/02/13/us/politics/mayorkas-impeachment-vote-trial.html',
 'https://www.nytimes.com/live/2024/02/14/world/israel-hamas-war-gaza-news',
 'https://www.nytimes.com/interactive/2023/10/07/world/middleeast/israel-gaza-maps.html',
 'https://www.nytimes.com/article/israel-gaza-photos.html',
 'https://www.nytimes.com/interactive/2024/02/01/

In [105]:
prefix = r'https://www.financialexpress.com/business/'

In [106]:
business = []
for article in fe.articles:
    if article.url.startswith(prefix):
        business.append(article)

In [107]:
len(business)

135

In [100]:
for article in business[:10]:
    print(article)

https://www.financialexpress.com/business/airlines-aviation-indigos-fleet-expansion-four-a320neo-aircraft-secured-in-boc-aviation-deal-all-about-engine-design-amp-enhanced-travel-experience-3393837/
https://www.financialexpress.com/business/airlines-aviation-aviation-sectors-upward-trajectory-crisil-predicts-over-20-surge-in-operating-profit-for-domestic-airlines-fleet-expansion-in-focus-3393556/
https://www.financialexpress.com/business/infrastructure-rites-ad-ports-group-sign-pact-for-infrastructure-development-3394219/
https://www.financialexpress.com/business/industry-ethos-limited-posts-q3-profit-growth-of-23-5-to-rs-2552-cr-revenue-up-224-despite-shradh-chennai-floods-store-renovation-3394273/
https://www.financialexpress.com/business/brandwagon-icubeswire-appoints-nivedita-dwivedi-as-the-senior-business-director-3393982/
https://www.financialexpress.com/business/industry-natco-pharma-q3-profit-jumps-241-4-to-rs-21270-crore-interim-dividend-announced-3394169/
https://www.financia

In [112]:
for article in business[:2]:
    article.download()
    article.parse()
    print(article.title, end='\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*\n')
    print(article.text, end='\n---------------------------------------------\n')

IndiGo’s Fleet Expansion: Four A320NEO aircraft secured in BOC aviation deal – All about engine design & enhanced travel experience
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
BOC Aviation Ltd, headquartered in Singapore, announced on Wednesday that it has finalized a finance lease agreement with InterGlobe Aviation Ltd (IndiGo) for four Airbus A320NEO aircraft. Steven Townend, CEO and Managing Director of BOC Aviation, expressed satisfaction with the deal, stating, “We are pleased to be closing another four finance leases with IndiGo. We continue to work closely with our long-time customer to support its expansion strategy as it builds a fleet of the latest technology fuel-efficient aircraft.”

What kind of engine is installed in A320NEO aircraft?

All four aircraft are equipped with CFM LEAP-1A engines and are scheduled for delivery in 2024. Riyaz Peermohamed, Chief Aircraft Acquisition and Financing Officer of IndiGo highlighted the significance of these aircraft in supporting the company’s expa

In [136]:
article.meta_data['og']['title']

'IndiGo’s Fleet Expansion: Four A320NEO aircraft secured in BOC aviation deal – All about engine design & enhanced travel experience'

In [137]:
type(article.text)

str

In [7]:
for category in fe.category_urls():
    print(category)

https://www.financialexpress.com/entertainment
https://www.financialexpress.com/
https://www.financialexpress.com/videos
https://www.financialexpress.com/education-2
https://hindi.financialexpress.com
https://epaper.financialexpress.com
https://www.financialexpress.com/photos
https://www.financialexpress.com/world-news
https://www.financialexpress.com/defence
https://finlit.financialexpress.com
https://www.financialexpress.com/brandwagon
https://www.financialexpress.com/opinion
https://www.financialexpress.com/industry
https://financialexpress.com
https://www.financialexpress.com/auto
https://www.financialexpress.com/lifestyle
https://www.financialexpress.com/india-news
https://www.financialexpress.com/money
https://www.financialexpress.com/web-stories
https://www.financialexpress.com
https://www.financialexpress.com/economy
https://www.financialexpress.com/market
https://www.financialexpress.com/sports


In [8]:
for feed in fe.feed_urls():
    print(feed)

In [9]:
fe.brand

'financialexpress'

In [10]:
fe.description

'Business News Live: Find here latest business news and finance news, share market updates, live stock market news, IPO update, banking and insurance sector updates. Check out for business news today in india, and live BSE/NSE stock price updates at financialexpress.com.'

In [13]:
article = fe.articles[0]
article.download()

In [60]:
article.url

'https://www.financialexpress.com/market/indian-indices-nse-nifty-50-companies-list/'

In [15]:
article.parse()

In [59]:
print(article.text)

Major Milestones

The "Nifty 50" refers to the National Stock Exchange of India's benchmark index, Nifty 50. It comprises 50 of the large and liquid stocks traded on the NSE. The index has hit several major milestones through the years. Here are some of the significant milestones for the Nifty 50

Introduction of Nifty 50 : The Nifty 50 index was introduced by the National Stock Exchange of India (NSE) on April 22, 1996. It provided a benchmark for investors to track the performance of the Indian stock market.

Nifty at 1000: The Nifty 50 index started with a base value of 1000.

Nifty at 2,000: The Nifty scaled the first 1000 points in 8 years. It touched 2000 on December 2, 2004.

Nifty at 3,000: However it took the Nifty a little over a year to scale the next 1000 points from 2000 to 3000. It hit the 3000 mark on January 30, 2006.

Global Financial Crisis: Just like all other key global indices, the index faced a major setback during the global financial crisis in 2008. It fell sign

In [17]:
article.authors

[]

In [18]:
article.title

'Nifty 50 (NSE) Index Price Live - Top Nifty 50 Companies Index Price in India Today, (NSE) India Index Price Live'

In [23]:
article.keywords

['nifty',
 'companies',
 'hit',
 '2000',
 '1000',
 'india',
 'live',
 'touched',
 'financial',
 'nse',
 'index',
 'price',
 '50',
 '3000',
 'mark',
 'today']

In [24]:
newspaper.hot()

['Lakers',
 'Mardi Gras',
 'Madame Web',
 'Real Madrid',
 'George Santos',
 'Stock Market Today',
 'Celtics',
 'Ash Wednesday',
 'RB Leipzig  Real Madrid',
 "Valentine's Day",
 'Paczki',
 'Mayorkas impeachment',
 "Galentine's Day",
 'Ash Wednesday 2024',
 'Peso Pluma',
 'Sun Day Red',
 'King cake',
 'Bucks',
 'Jaafar Jackson',
 'IHOP free pancakes National Pancake Day']

In [25]:
newspaper.popular_urls()

['http://www.huffingtonpost.com',
 'http://cnn.com',
 'http://www.time.com',
 'http://www.ted.com',
 'http://pandodaily.com',
 'http://www.cnbc.com',
 'http://www.mlb.com',
 'http://www.pcmag.com',
 'http://www.foxnews.com',
 'http://theatlantic.com',
 'http://www.bbc.co.uk',
 'http://www.vice.com',
 'http://www.elle.com',
 'http://www.vh1.com',
 'http://espnf1.com',
 'http://espn.com',
 'http://www.npr.org',
 'http://www.sfgate.com',
 'http://www.glamour.com',
 'http://www.whosdatedwho.com',
 'http://kotaku.com',
 'http://thebostonchannel.com',
 'http://www.suntimes.com',
 'http://www.businessinsider.com',
 'http://www.rivals.com',
 'http://thebusinessjournal.com',
 'http://www.newrepublic.com',
 'http://allthingsd.com',
 'http://www.topgear.com',
 'http://thecitizen.com',
 'http://www.ign.com',
 'http://www.sci-news.com',
 'http://www.morningstar.com',
 'http://www.variety.com',
 'http://thebottomline.as.ucsb.edu',
 'http://www.gamefaqs.com',
 'http://blog.searchenginewatch.com',
 'h

In [ ]:

PERSIST_DIR = "./storage"
storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
#storage_context.docstore.add_documents(nodes)
index = load_index_from_storage(storage_context)
nodes = index.docstore.docs.values()

nodes = list(nodes)

In [ ]:
prompt = '''
Outline the financial and operational highlights of Meta for Q4:2023 and full year 2023 from the source documents. Include details like revenue, profits, share prices, user base, etc./
You have to prove that your response is correct by citing the relevant sections from the source document./
Cross-check your response for factual accuracy and correct it, if needed. Your response must not contain any information that is not present in the source document./
Structure your output as a paragraph under 300 words. FOLLOW ALL THE INSTRUCTIONS CAREFULLY.'''

retrieved_nodes = hybrid_retriever.retrieve(prompt)

reranked_nodes = reranker.postprocess_nodes(
    retrieved_nodes,
    query_bundle=QueryBundle(prompt)
)

  Based on the provided source documents, here are the financial and operational highlights of Meta for Q4:2023 and full year 2023:
For Q4:2023, Meta reported revenue of $40,111 million, a 25% increase year-over-year. The company's operating margin was 41%, and its effective tax rate was 17%. Net income was $14,017 million, a 203% increase year-over-year, and diluted earnings per share (EPS) were $5.33, a 203% increase. Family daily active people (DAP) increased by 8% year-over-year to an average of 3.19 billion for the quarter.
For full year 2023, Meta reported revenue of $134,902 million, a 16% increase year-over-year. The company's operating margin was 35%, and its effective tax rate was 18%. Net income was $39,098 million, a 69% increase year-over-year, and diluted EPS were $14.87, a 73% increase. User base also grew with Family monthly active people (MAP) increasing by 6% year-over-year to 3.07 billion as of December 31, 2023, and Facebook daily active users (DAUs) increasing by 6% year-over-year to 2.11 billion on average for the quarter.
In terms of costs and expenses, total expenses were $23,727 million for Q4:2023, a decrease of 8% year-over-year, and $88,151 million for full year 2023, an increase of 1% year-over-year. Restructuring charges amounted to $1.15 billion and $3.45 billion for Q4:2023 and full year 2023, respectively.
Finally, as of December 31, 2023, Meta had $30.93 billion available and authorized for share repurchases, with an additional $50 billion increase in its share repurchase authorization announced today.

## Response Synthesis

* o/p of a response synthesizer is a Response object
* when used in a query engine, the response synthesizer is used after nodes are retrieved from a retriever and after any node-postprocessors are ran
* response modes : refine, compact (default), tree_summarize, simple_summarize, no_text, accumulate, compact_accumulate
* with refine/compact, you can use structured_answer_filtering=True in get_response_synthesizer()

In [ ]:
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodewithScore
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core import get_response_synthesizer

# on its own
response_synthesizer = get_response_synthesizer(
    'query text',
    nodes = [Node(text='text'), ...],
    response_mode = 'compact'
)

# in a query engine
query_engine = index.as_query_engine(
    response_synthesizer=response_synthesizer
)
response = query_engine.query('query_text')

In [ ]:
########################################
############ TREE-SUMMARIZE ############
########################################

from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(verbose=True)
response = await summarizer.aget_response('query', [text])
print(response)

In [ ]:
########################################
####### PYDANTIC TREE-SUMMARIZE ########
########################################

from llama_index.core.types import BaseModel
from llama_index.core import PromptTemplate
from typing import List

class Biography(BaseModel):
    name: str
    best_known_for: List(str)
    extra_info: str

qa_prompt_tmpl = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Please also write the answer in the style of {tone_name}.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt = PromptTemplate(qa_prompt_tmpl)

summarizer = TreeSummarize(
    verbose=True,
    summary_template=qa_prompt,
    output_cls=Biography
)

response = summarizer.get_response(
    'query_text',
    [text],
    tone_name='tone'
)

print(str(response))

# Readers

In [ ]:
# reader usage
from llama_index.core import download_loader

GoogleDocsReader = download_loader('GoogleDocsReader')
loader = GoogleDocsReader()
documents = loader.load_data(document_ids=gdoc_ids)

In [ ]:
# PyMuPDF reader
from pathlib import Path
PyMuPDFReader = download_loader('PyMuPDFReader')
loader = PyMuPDFReader()
documents = loader.load_data(
    file_path=Path('./article.pdf'),
    metadata=True
)

In [ ]:
# PDF table loader
from llama_hub.pdf_table import PDFTableReader
from pathlib import Path

reader = PDFTableReader()
pdf_path = Path('./path/to/pdf')
documents = reader.load_data(
    file=pdf_path,
    pages='all' # '80-90', '1,2,3'
)

In [ ]:
# SEC data downloader
SECFilingsLoader = download_loader('SECFilingsLoader')
loader = SECFilingsLoader(
    tickers=['TSLA'],
    amount=3,
    filing_type='10-K'
)
loader.load_data()
documents = SimpleDirectoryReader('data\TSLA\2022').load_data()

In [ ]:
from llama_index import VectorStoreIndex, download_loader
from llama_index import SimpleDirectoryReader

SECFilingsLoader = download_loader("SECFilingsLoader")

loader = SECFilingsLoader(tickers=["TSLA"], amount=3, filing_type="10-K")
loader.load_data()

documents = SimpleDirectoryReader("data\TSLA\2022").load_data()
index = VectorStoreIndex.from_documents(documents)
index.query("What are the risk factors of Tesla for the year 2022?")

In [ ]:
# SmartPDFLoader
%pip install llmsherpa

from llama_hub.smart_pdf_loader import SmartPDFLoader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "https://arxiv.org/pdf/1910.13461.pdf"  # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_loader = SmartPDFLoader(llmsherpa_api_url=llmsherpa_api_url)
documents = pdf_loader.load_data(pdf_url)

# summarize a section using prompts
from IPython.core.display import display, HTML
selected_section = None
# find a section in the document by title
for section in doc.sections():
    if section.title == '3 Fine-tuning BART':
        selected_section = section
        break
# use include_children=True and recurse=True to fully expand the section. 
# include_children only returns at one sublevel of children whereas recurse goes through all the descendants
HTML(section.to_html(include_children=True, recurse=True))
# summary
from llama_index.llms import OpenAI
context = selected_section.to_html(include_children=True, recurse=True)
question = "list all the tasks discussed and one line about each task"
resp = OpenAI().complete(f"read this text and answer question: {question}:\n{context}")
print(resp.text)

In [ ]:
# analyze tables
from IPython.core.display import display, HTML
HTML(doc.tables()[5].to_html())

from llama_index.llms import OpenAI
context = doc.tables()[5].to_html()
resp = OpenAI().complete(f"read this table and answer question: which model has the best performance on squad 2.0:\n{context}")
print(resp.text)

In [ ]:
# news
%pip install newspaper3k
from llama_index.readers.web.news import NewsArticleReader

reader = NewsArticleReader(use_nlp=False)
documents = reader.load_data(
    [
        "https://www.cnbc.com/2023/08/03/amazon-amzn-q2-earnings-report-2023.html",
        "https://www.theverge.com/2023/8/3/23818388/brave-search-image-video-results-privacy-index",
    ]
)

In [ ]:
# youtube video transcripts
%pip install youtube_transcript_api

from llama_hub.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=i3OYlaoj-BM"]
)

# check url validity
from llama_hub.youtube_transcript import is_youtube_video

is_youtube_video("https://youtube.com/watch?v=j83jrh2")  # => True
is_youtube_video("https://vimeo.com/272134160")  # => False

In [ ]:
# Summary index
from llama_index.core import SummaryIndex
summary_index = SummaryIndex()

## Customizing Prompts

In [ ]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

query_engine = index.as_query_engine()

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [ ]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**response_synthesizer:text_qa_template**

```
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 
```

**response_synthesizer:refine_template**

```
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
```

**response_synthesizer:summary_template**
```
Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge, answer the query.
Query: {query_str}
Answer: 
```

In [ ]:
# customize the prompts
new_qa_template = (
    "You are an experienced financial and investment research analyst, with profound analytical capabilities."
    "Your task is to write high-quality financial and investment research reports, which are to be published by an esteemed firm, with a large user base."
    "You perform in-depth research about a company, the industry and its capabilities, in light of the current financial condition."
    "You always generate your response based on the context information from the source documents, given below, delineated by triple backticks (```) ONLY.\n"
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "```{context_str}```"
    "\n-----------------------------------------------------------------------------------------------------------------------\n"
    "Given this context information. please answer the question: {query_str}\n"
    "Cite the relevant context sources that you use to answer the question. You use the numerical facts and data from the context documents to elucidate your answer."
    "YOUR RESPONSE MUST NOT INCLUDE ANY DATA THAT IS NOT PRESENT IN THE SOURCE DOCUMENTS. Your response should be credible and trustworthy."
)

new_refine_template = (
    "You are an profoundly experienced document curator, excellent at reviewing and refining the research articles written by financial and investment research analysts."
    "You refine the existing articles to make them excellent enough to be published in a research report."
    "The original query is as follows: {query_str}"
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "We have provided an existing article written by the analyst: {existing_answer}"
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "You have the opportunity to refine the existing article to make it even better (only if needed) with some more context given below.\n"
    "{context_msg}"
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "Given the new context, cross-check the original article for factual accuracy and relevance. Refine it so that it looks more professional and crisp."
    "The refined answer must be of high-quality, apt to be included in a research report. If the context isn't useful, return the original answer."
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "Refined Answer:"
)

new_summary_template = (
    "You are an experienced financial and investment research analyst, with profound analytical capabilities."
    "Your task is to summarize the financial documents given below as context, delineated by triple backticks (```), including every important detail that is relevant"
    "for including in a financial and investment research report. You ONLY USE THE INFORMATION GIVEN IN THE CONTEXT AND STRICTLY AVOID HALLUCINATING ANYTHING."
    "Your summary should be in-depth and must be relevant in light of the current financial condition.\n"
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "```{context_str}```"
    "-----------------------------------------------------------------------------------------------------------------------\n"
    "Using the context and the summaries that you generate, AND NOT PRIOR KNOWLEDGE, answer the query."
    "Query: {query_str}"
    "Answer:" 
)

new_qa_prompt = PromptTemplate(new_qa_template)
new_qa_prompt = PromptTemplate(new_qa_template)
new_summary_prompt = PromptTemplate(new_summary_template)
query_engine.update_prompts(
    {
        'response_synthesizer:text_qa_template': new_qa_prompt
    }
)
